# Random Forest

Training:
- Get a random subset of the dataset
- Create a decision tree with this random subset.
- Repeat for as many times as the number of trees that we want.

Testing (given a datapoint we want to test)
- Get the predictions from each tree
- Hold a majority vote

In [1]:
import numpy as np
import pandas as pd
np.random.seed(42)

In [2]:
def most_common_label(y):
    '''
    Find the most common label in a list of labels.
    '''
    counts = {}
    for label in y:                 
        counts[label] = counts.get(label, 0) + 1
    return max(counts, key=counts.get)

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None): #value will only be passed to leaf nodes
        '''
        Initialize a node in the decision tree.
        
        Parameters
        ----------
        feature : int
            Index of the feature used for splitting
        threshold : float
            Threshold value for splitting
        left : Node
            Left child node
        right : Node
            Right child node
        value : int
            Value for leaf nodes (class label)
        '''
        self.feature = feature
        self.threshold = threshold # The split value for a decision node
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

class DecisionTree:
    
    def __init__(self, min_samples_split=3, max_depth=10, n_features=None):
        '''
        Initialize a decision tree classifier.
        
        Parameters
        ----------
        min_samples_split : int 
            Minimum number of samples required to split an internal node
        max_depth : int 
            Maximum depth of the tree
        n_features : int 
            Number of features considered when looking for the best split
        '''
       
        self.min_samples_split = min_samples_split 
        self.max_depth = max_depth
        self.n_features = n_features 
        self.root = None

    def fit(self, X, y):
        '''
        Build a decision tree classifier from the training set (X, y).
        '''
        if not self.n_features:
            self.n_features = X.shape[1]
        else:
            self.n_features = min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X,y)

    # Recursive function to grow the tree
    def _grow_tree(self, X, y, depth=0):
        '''
        Recursively grow the decision tree.
            
        Returns
        -------
        Node
            Root node of the grown tree
        '''
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # Stopping criteria
        if (depth >= self.max_depth or
            n_samples < self.min_samples_split or
            n_labels == 1):
            leaf_value = most_common_label(y)
            return Node(value=leaf_value)
        
        subset_features = np.random.choice(n_features, self.n_features, replace=False) # Only use a subset of unique features

        # Find the best split
        best_thresh, best_feature = self._best_split(X, y, subset_features)
        
        # Create the left and right subtrees

        left_idxs, right_idxs = self._split(X[:,best_feature],best_thresh)

        if len(left_idxs)  == 0 or len(right_idxs) == 0:
            leaf_value = most_common_label(y)
            return Node(value=leaf_value)

        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    # Finding the best splits and thresholds 
    def _best_split(self, X, y, subset_features):
        '''
        Find the best feature and threshold to split on.

        Returns
        -------
        tuple
            (best_threshold, best_feature_index)
        '''
        best_gain = -1
        split_idx, split_thresh = None, None
        for feature in subset_features:
            X_column = X[:, feature]
            thresholds = np.unique(X_column)

            for threshold in thresholds:
                # Calculate the information gain
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feature
                    split_thresh = threshold
        return split_thresh, split_idx


    def _information_gain(self,y,X_column, threshold):
        '''
        Calculate the information gain of a split:
        E(parent) - [weighted average] * E(children)
        '''

        parent_entropy = self._entropy(y)

        left_idxs, right_idxs = self._split(X_column, threshold)

        if(len(left_idxs) == 0 or len(right_idxs) == 0):
            return 0

        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n)*e_l + (n_r/n)*e_r

        information_gain = parent_entropy - child_entropy
        return information_gain
    

    def _split(self, X_column, split_thresh):
        '''
        Split the data based on a threshold value.

        Returns
        -------
        tuple
            (left_indices, right_indices)
        '''
        left_idxs = np.argwhere(X_column<=split_thresh).flatten() 
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs
    def _entropy(self,y):
        '''
        Calculate the entropy of a set of labels:
        - Sum(p(X)*log_2(p(X)))
        '''
        # Creates a historgram [instances_of_0, instances_of_1, ...]
        hist = np.bincount(y)
        ps = hist/len(y)
        
        # Only keep probabilities > 0
        ps_nz = ps[ps > 0]
        return -np.sum(ps_nz * np.log(ps_nz)) 

    

    def _traverse_tree(self, x, node):
        '''
        Traverse the tree to make a prediction for a single sample.
        Parameters
        ----------
        x : array-like of shape of features
            Single sample to predict
        node : Node
            Current node in the tree. The process is recursive
            
        Returns
        -------
        int or str
            Predicted class label

        '''
        if node.is_leaf_node():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x,node.left)
        return self._traverse_tree(x,node.right)

    
    def predict(self, X):
        '''
        Predict class labels for samples in X.
        '''
        return np.array([self._traverse_tree(x,self.root) for x in X])
    
    

In [3]:
class RandomForest():
    def __init__(self, n_trees=200, max_depth=10, min_samples_split=3, n_features=None):
        '''
        Initialize a random forest classifier.
        
        Parameters
        ----------
        n_trees : int, optional
            Number of trees in the forest
        max_depth : int, optional
            Maximum depth for each tree
        min_samples_split : int, optional
            Minimum number of samples required to split an internal node
        n_features : int, optional
            Number of features to consider when looking for the best split
        '''
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.trees = []


    def fit(self, X,y):
        '''
        Build a random forest classifier from the training set (X, y).
        '''

        n_tot_features = X.shape[1]
        if self.n_features is None:
            self.n_features = int(np.sqrt(n_tot_features))
        else:
            self.n_features = min(self.n_features, n_tot_features) 

        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(max_depth=self.max_depth,
                        min_samples_split=self.min_samples_split,
                         n_features=self.n_features)
            X_sample, y_sample =self._bootstrap_samples(X,y)
            tree.fit(X_sample,y_sample)
            self.trees.append(tree)
        return self

    def _bootstrap_samples(self,X,y):
        '''
        Create bootstrap samples from the training data.
        Returns
        -------
        tuple
            (X_bootstrap, y_bootstrap)
        '''
        n_samples, _ = X.shape
        idxs = np.random.choice(n_samples, n_samples, replace=True) # sampling with replacement
        return X[idxs], y[idxs]
        
    
    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees]) # [[pred_1st_sample_by_1st_tree, pred_2st_sample_by_1st_tree],
                                                                        #   [pred_1st_sample_by_2nd_tree],[pred_2nd_sample_by_2nd_tree],
                                                                        #   [...], ...]
        
        #[[all predictions of first sample], [all predictions of second sample], [...], ...]
        tree_predictions = np.swapaxes(predictions, 0, 1)
        predictions = np.array([most_common_label(pred) for pred in tree_predictions])
        return predictions

In [4]:

# df1 = pd.read_csv('data/GenreClassData_30s.txt', sep='\t')
# df2 = pd.read_csv('data/GenreClassData_10s.txt', sep='\t')
# df3 = pd.read_csv('data/GenreClassData_5s.txt', sep='\t')

# frames = [df1, df2, df3]
# result = pd.concat(frames)
# print(f"{len(result)} rows in the result, {len(df1)} in df1, {len(df2)} in df2, {len(df3)} in df3")


In [5]:

from sklearn import preprocessing

result = pd.read_csv('data/GenreClassData_30s.txt', sep='\t')

features = [col for col in result.columns if col not in ['Track ID', 'File', 'GenreID', 'Genre', 'Type']]
targets = ['GenreID'] 

# Split the data into training and testing sets
train = result[result['Type'] == 'Train'].copy()
test = result[result['Type'] == 'Test'].copy()
# feature data
X_train, y_train = train[features], train[targets]
X_test, y_test = test[features], test[targets]

min_max_scaler = preprocessing.MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)

y_train = y_train.to_numpy().ravel()  # flater ut fra (n,1) til (n,)
y_test  = y_test.to_numpy().ravel()

X_test.view()
X_train.view()

array([[0.43438696, 0.27287016, 0.70602324, ..., 0.25229837, 0.13490084,
        0.1865418 ],
       [0.30832523, 0.2935099 , 0.73328814, ..., 0.41676686, 0.21867656,
        0.38473969],
       [0.29358315, 0.3303101 , 0.40756804, ..., 0.38169812, 0.2802361 ,
        0.31613867],
       ...,
       [0.42687408, 0.22619052, 0.39196788, ..., 0.11390357, 0.06748416,
        0.10959762],
       [0.09678806, 0.10171028, 0.10111576, ..., 0.21896671, 0.15575828,
        0.38902993],
       [0.2628941 , 0.1523535 , 0.51456561, ..., 0.35766246, 0.27377917,
        0.57002067]], shape=(792, 63))

In [6]:
clf = RandomForest()
clf.fit(X_train, y_train)

In [ ]:
def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

predictions = clf.predict(X_test)
acc = accuracy(y_test, predictions)

print(acc)

0.7272727272727273


In [8]:

from sklearn.metrics import classification_report

classes = np.unique(train['GenreID'])
print("Per‐class performance:")
print(classification_report(
    y_test,
    predictions, 
    labels=classes,       
    digits=4,
))

Per‐class performance:
              precision    recall  f1-score   support

           0     0.7083    0.8500    0.7727        20
           1     0.8000    1.0000    0.8889        20
           2     0.4211    0.4000    0.4103        20
           3     0.7619    0.8000    0.7805        20
           4     0.7727    0.8500    0.8095        20
           5     0.9412    0.8421    0.8889        19
           6     0.5000    0.3000    0.3750        20
           7     0.7727    0.8500    0.8095        20
           8     0.6667    0.6316    0.6486        19
           9     0.8333    0.7500    0.7895        20

    accuracy                         0.7273       198
   macro avg     0.7178    0.7274    0.7173       198
weighted avg     0.7169    0.7273    0.7168       198

